In [ ]:
!pip install transformers datasets accelerate torch

In [ ]:
# fine_tune_letter_generator.py

from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
import torch

# 1. Подготовка данных (примеры писем)
letters = [
    "Уважаемый Иван Петрович,\n\nБлагодарим вас за сотрудничество в прошлом месяце. Мы рады сообщить, что ваш заказ успешно обработан и будет отправлен в ближайшие дни.\n\nС уважением,\nКоманда поддержки",
    "Здравствуйте, Анна!\n\nНапоминаем, что завтра состоится наша онлайн-встреча в 15:00 по московскому времени. Ссылка для подключения будет отправлена за 10 минут до начала.\n\nСпасибо!",
    "Добрый день!\n\nМы получили ваш запрос и в ближайшее время с вами свяжемся. Пожалуйста, проверяйте почту и спам.\n\nС уважением,\nОтдел клиентского сервиса",
    # Добавьте больше примеров писем здесь
]

# 2. Создание датасета
dataset_dict = {"text": letters}
dataset = Dataset.from_dict(dataset_dict)

# 3. Загрузка токенизатора и модели
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"  # или "gpt2" для английского
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Добавляем специальный токен padding, если его нет
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# 4. Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# 5. Data collator для маскированного языкового моделирования
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False  # GPT-2 не использует MLM
)

# 6. Настройка обучения
training_args = TrainingArguments(
    output_dir="./letter_model",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=100,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  # отключаем интеграции вроде Weights & Biases
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)

# 7. Обучение
trainer.train()

# 8. Сохранение модели
model.save_pretrained("./letter_model")
tokenizer.save_pretrained("./letter_model")

# 9. Пример генерации нового письма
def generate_letter(prompt: str, max_length=150):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs.input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Пример использования
print(generate_letter("Уважаемый клиент,\n\n"))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,2.882600
20,0.633900


Уважаемый клиент,

С уважением, Администрация
Получите письмо с подтверждением.
Я внимательно проверяла ваш заказ, в течение недели. С уважением
Благодарим вас за сотрудничество в прошлом месяце. Мы рады сообщить, что ваш отзыв успешно обработан и будет отправлен в ближайшее время. Пожалуйста, проверяйте почту и спам. В ближайшее рабочее время с вами будет отправлена ссылка для подтверждения. Спасибо!
Мы отправили ваш запрос и в ближайшие дни с ним будет обработан. Передайте ваш вопрос через специальную ссылку в 15 минут. Надеемся, и через несколько минут ссылка будет доставлена. Откат и адрес вашей заявки будет автоматически отправлен. Поздравляем вас с в ближайшие 10 минут
Спасибо за ваш положительный отзыв. Если


In [ ]:
# Подключаем Google Drive (если используете Colab)
from google.colab import drive
drive.mount('/content/drive')

import os
import glob

# Путь к папке с письмами
folder_path = "/content/drive/MyDrive/AI/письма/letters/"

# Находим все .txt файлы
txt_files = glob.glob(os.path.join(folder_path, "*.txt"))

if not txt_files:
    print("❌ В папке нет файлов с расширением .txt")
else:
    print(f"Найдено {len(txt_files)} файлов. Начинаю перекодировку из cp1251 в UTF-8...\n")

    for file_path in txt_files:
        filename = os.path.basename(file_path)
        try:
            # Читаем файл в исходной кодировке Windows-1251
            with open(file_path, 'r', encoding='cp1251') as f:
                content = f.read()

            # Перезаписываем в UTF-8 (оригинал заменяется)
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(content)

            print(f"✅ {filename} — успешно перекодирован")

        except UnicodeDecodeError:
            print(f"⚠️ {filename} — ошибка декодирования (возможно, уже в UTF-8 или другой кодировке)")
        except Exception as e:
            print(f"❌ {filename} — ошибка: {str(e)}")

    print("\n✅ Все файлы обработаны. Теперь они в кодировке UTF-8 и готовы для использования в ML.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Найдено 4 файлов. Начинаю перекодировку из cp1251 в UTF-8...

✅ статус 08082025.txt — успешно перекодирован
✅ статус 15082025.txt — успешно перекодирован
✅ статус 20082025.txt — успешно перекодирован
✅ статус 26092025.txt — успешно перекодирован

✅ Все файлы обработаны. Теперь они в кодировке UTF-8 и готовы для использования в ML.


In [1]:
# fine_tune_letter_generator_from_files.py

import os
from pathlib import Path
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from datasets import Dataset

# 1. Укажите путь к папке с письмами
LETTERS_FOLDER = "/content/drive/MyDrive/AI/письма/letters"  # ← измените, если папка называется иначе

# 2. Загрузка всех .txt файлов из папки
def load_letters_from_folder(folder_path):
    letters = []
    folder = Path(folder_path)
    if not folder.exists():
        raise FileNotFoundError(f"Папка {folder_path} не найдена!")

    for txt_file in folder.glob("*.txt"):
        try:
            with open(txt_file, "r", encoding="utf-8") as f:
                content = f.read().strip()
                if content:  # пропускаем пустые файлы
                    letters.append(content)
        except Exception as e:
            print(f"Ошибка при чтении {txt_file}: {e}")

    print(f"Загружено {len(letters)} писем из папки '{folder_path}'")
    return letters

# 3. Загружаем письма
letters = load_letters_from_folder(LETTERS_FOLDER)

if not letters:
    raise ValueError("Нет писем для обучения! Проверьте папку и формат файлов.")

# 4. Создаём датасет
dataset_dict = {"text": letters}
dataset = Dataset.from_dict(dataset_dict)

# 5. Загрузка модели и токенизатора (для русского языка)
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"  # или "gpt2" для английского
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Добавляем pad_token, если его нет
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# 6. Токенизация
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=256  # можно увеличить, если письма длинные
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

# 7. Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 8. Настройка обучения
training_args = TrainingArguments(
    output_dir="./letter_model",
    overwrite_output_dir=True,
    num_train_epochs=250,
    per_device_train_batch_size=2,
    save_steps=100,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)

# 9. Обучение
trainer.train()

# 10. Сохранение
model.save_pretrained("./letter_model")
tokenizer.save_pretrained("./letter_model")

# 11. Функция генерации
def generate_letter(prompt: str, max_length=300):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs.input_ids,
        max_length=max_length,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Пример использования
if __name__ == "__main__":
    example_prompt = "Уважаемый клиент,\n\n"
    print("Сгенерированное письмо:\n")
    print(generate_letter(example_prompt))

Загружено 4 писем из папки '/content/drive/MyDrive/AI/письма/letters'


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,1.934200
20,0.518400
30,0.130600
40,0.047600
50,0.030000
60,0.018900
70,0.012100
80,0.009100
90,0.010100
100,0.005600


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument i

Сгенерированное письмо:

Уважаемый клиент,

С администрацией Курортного района согласованы точки подключения к сетям 220 и места хранения ТМЦ.
Получены планы в масштабе 1:500 по объектам Новый пляж и Чудный пляж.

Определены плановые даты начала подготовительных работ по пляжам: 
Пляж Сестрорецкий 15.10.2025
Пляж Дюны 1 начаты подготовительные работы 20.08.2025, разработка траншей 21.08.2025
Пляж Дюны 2 (Белые ночи) начаты подготовительные работы 20.08.2025, разработка траншей 21.08.2025
Пляж Офицерский выполены подготовительные работы, разработка траншей 20.08.2025
Пляж Северный выполены подготовительные работы, разработка траншей 20.08.2025

Процесс производства работ.
Пляж Ласковый выполнены подготовительные работы, осуществлен завоз ТМЦ
Пляж Золотой выполнены подготовительные работы, завоз ТМЦ, разработаны траншеи, 
произведена укладка кабельной канализации, монтаж опор и кабеля запланирован на 25.08.2025.

Процесс производства работ.
Пляж Золотой выполнены подготовительные работы,

In [ ]:
import chardet

# Определяем кодировку
with open('/content/drive/MyDrive/AI/письма/letters/статус 15082025.txt', 'rb') as f:
    raw_data = f.read()
    encoding = chardet.detect(raw_data)['encoding']
    print(f"Обнаруженная кодировка: {encoding}")

# Читаем с правильной кодировкой
text = raw_data.decode(encoding)
print(text)

Обнаруженная кодировка: utf-8
Видеонаблюдение и видеоаналитика на объектах Курортного района — обновление на 15.08.2025

Завершено обследование всех ранее представленных объектов.

С администрацией Курортного района согласованы точки подключения к сетям 220 и места хранения ТМЦ.
Получены планы в масштабе 1:500 по объектам Новый пляж и Чудный пляж.

Определены плановые даты начала подготовительных работ по пляжам: 
Пляж Сестрорецкий 15.10.2025
Пляж Дюны 1 30.09.2025
Пляж Дюны 2 (Белые ночи) 15.10.2025
Пляж Офицерский 15.09.2025
Пляж Северный 18.08.2025, при этом завоз ТМЦ на пляж Северный 30.08.2025.

Процесс производства работ.
Пляж Ласковый выполнены подготовительные работы, осуществлен завоз ТМЦ
Пляж Золотой выполнены подготовительные работы, завоз ТМЦ, разработаны траншеи, 
произведена укладка кабельной канализации, монтаж опор и кабеля запланирован на 25.08.2025.

